<a href="https://colab.research.google.com/github/dharmesh-b/AI-for-Finance-Stocks-real-time-analysis-/blob/master/hb_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Workaround to avoid following error at notebook
# Error: NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U huggingface_hub

#LangChain related libraries
!pip install -q -U langchain

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf

#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers

# FAISS Vector Databses specific Libraries
!pip install -q -U faiss-gpu

In [ ]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

**We are using Mistral 7 Billion parameter LLM (Open source from HuggingFace)**
If you're new to Hugging Face or machine learning tasks, the following lines of code introduce some scary concepts. This Mistral 7B LLM, although big in size (approximately 14 GB), presents a challenge when loading on a Colab notebook, requiring larger GPUs and making it incompatible with the free T4 GPU.

To address this issue, we employ a sharded version of the Mistral LLM. This version is loaded in smaller increments (approximately 1.9 GB each) in the notebook and subsequently aggregated into a single file. This allows us to seamlessly utilize the LLM on a free Colab GPU (T4 GPU) without incurring any costs for learning purposes.

These lines of code incorporate several important concepts, which may initially seem complex but are easily comprehensible:

BitsAndBytesConfig:
load_in_4bit
bnb_4bit_use_double_quant
bnb_4bit_quant_type
bnb_4bit_compute_dtype
AutoModelForCausalLM.from_pretrained

AutoTokenizer.from_pretrained

Despite the use of some technical terminology, don't be discouraged. In the coming days, we'll delve into these concepts in an approachable manner, breaking down the complexities. This includes a detailed understanding of the mentioned configurations, loading procedures, and tokenization processes. Embrace the learning journey; it's simpler than it seems.

In [ ]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (origin_model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

Creating pipelines to run LLM at Colab notebook
It uses

Huggingface transformers pipeline
This is an object that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.

We are using "text-generation" task. It's sort of chatGPT where you ask question and model respond with answer of that question based on it's intelligence.

LangChain HuggingFacePipeline
Integrating "transformers.pipeline" with LangChain.
You may be thinking why we need this. We will be using LangChain extensively later and this is first step to integrate our LLM with LangChain

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature = 0.3,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Using CSV File with LangChain CSVLoader

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
# Load the CSV file
loader = CSVLoader(file_path='/content/drive/MyDrive/Colab Notebooks/pune_house_prices.csv')
data = loader.load()

# Split the documents into smaller chunks
#separator=""
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs  = text_splitter.split_documents(data)

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

In [ ]:
#We will run an infinite loop to ask questions to LLM and retrieve answers untill the user wants to quit
import sys
chat_history = []
query = input('Prompt: ')
# Ask question related CSV file like Which country has highest green gas?
result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))